In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
import json
import sys
import torch
import time

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
batch_size = 10000

In [5]:
distilroberta = SentenceTransformer('stsb-distilroberta-base-v2', device=device)
distilroberta.max_seq_length = 512

usenc = SentenceTransformer('distiluse-base-multilingual-cased-v1', device=device)
usenc.max_seq_length = 512

In [ ]:
filename = 'segmentaaa-output'

In [9]:
start = time.time()
with open(filename) as f:
  with open(filename + '_with_embeddings', 'w') as fout:
    docs = []
    datas = []
    count = 0

    for line in f:
        doc = json.loads(line)
        docs.append(doc)
        datas.append(doc['_source']['content_t'])

        count += 1
        if count == batch_size:
            use_embeddings = usenc.encode(datas, device=device)
            distilroberta_embeddings = distilroberta.encode(datas, device=device)
  
            for idx, doc in enumerate(docs):
                doc['_source']['use_embedding'] = use_embeddings[idx].tolist()
                doc['_source']['distilroberta_embedding'] = distilroberta_embeddings[idx].tolist()
                fout.write(json.dumps(doc) + '\n')
            datas = []
            docs = []
            count = 0
    
    if count > 0:
        use_embeddings = usenc.encode(datas, device=device)
        distilroberta_embeddings = distilroberta.encode(datas, device=device)
  
        for idx, doc in enumerate(docs):
            doc['_source']['use_embedding'] = use_embeddings[idx].tolist()
            doc['_source']['distilroberta_embedding'] = distilroberta_embeddings[idx].tolist()
            fout.write(json.dumps(doc) + '\n')
        
            
print(time.time() - start)

7821.51424574852
